In [1]:
import sys
sys.path.append('../')
import numpy as np
import pandas as pd
from grid_world.grid_world import GridWorld
from DMEIRL.DeepMEIRL_FC import DeepMEIRL_FC
from grid_world.trajGen_grid_world import GridWorld_trajGen
from utils import utils

from grid_world.data_parser import DataParser
from grid_world.grid_world import GridWorld
import torch

import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2

no background image


In [2]:
parser = DataParser(width=10,height=10)
from PIL import Image
import os
from tqdm import tqdm
tracks_path = 'demo_label/train/'
file_names = os.listdir(tracks_path)
#read images
imgs = []
for file_name in file_names:
    imgs.append(Image.open(tracks_path + "/" + file_name))

#parse images
for i in tqdm(range(len(imgs))):
    parser.ParseEnvironmentFromImage(imgs[i],file_names[i].split('.')[0],save_path='demo_env&feature')

100%|██████████| 4/4 [00:00<00:00, 162.91it/s]


In [3]:
model = DeepMEIRL_FC(n_input=4,layers=(16,32,32,16))
model.to('cuda')
model.load_state_dict(torch.load('demo_result/1_model.pth'))
model.eval()
model.cuda()

DeepMEIRL_FC(
  (net): Sequential(
    (0): Linear(in_features=4, out_features=16, bias=True)
    (1): ELU(alpha=1.0)
    (2): Linear(in_features=16, out_features=32, bias=True)
    (3): ELU(alpha=1.0)
    (4): Linear(in_features=32, out_features=32, bias=True)
    (5): ELU(alpha=1.0)
    (6): Linear(in_features=32, out_features=16, bias=True)
    (7): ELU(alpha=1.0)
    (8): Linear(in_features=16, out_features=1, bias=True)
    (9): Sigmoid()
  )
)

In [4]:
n = np.nan
state_mat = np.array([[ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                       [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                       [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                       [ 0, 0, 0, n, n, 0, n, n, n, n],
                       [ 0, 0, 0, n, 0, 0, 0, 0, 0, 0],
                       [ 0, 0, 0, n, 0, 0, 0, 0, 0, 0],
                       [ 0, 0, 0, n, 0, 0, 8, 8, 8, 8],
                       [ 0, 0, 0, n, 0, 0, 8, 8, 8, 8],
                       [ 0, 0, 0, n, 0, 0, 8, 8, 8, 8],
                       [ 0, 0, 0, n, 0, 0, 8, 8, 8, 8],])

world_trajGen = GridWorld_trajGen(10,10,states_matrix=state_mat,
                                  features_folderPath='demo_env&feature/features_grid/0225_10x10',
                                  rewards_mul=[1,-2,-1,-1],
                                  init_states=list(range(0,10)),
                                  trans_prob=0.6,discount=0.9,
                                  model = model)

world_trajGen.ShowLearnedRewards()

In [5]:
world_trajGen.GenerateTrajectoriesWithLearnedReward(traj_count=500,traj_length=20)

100%|██████████| 500/500 [00:00<00:00, 3965.21it/s]


,m,trajs
0,1,"[(5, 1, 15), (15, 1, 25), (25, 1, 35), (35, 1,..."
1,2,"[(1, 1, 11), (11, 1, 21), (21, 1, 20), (20, 1,..."
2,3,"[(1, 1, 0), (0, 1, 10), (10, 1, 20), (20, 1, 2..."
3,4,"[(0, 1, 0), (0, 1, 0), (0, 1, 1), (1, 1, 11), ..."
4,5,"[(2, 1, 12), (12, 1, 22), (22, 1, 32), (32, 1,..."
...,...,...
495,496,"[(0, 1, 0), (0, 1, 10), (10, 1, 10), (10, 1, 1..."
496,497,"[(9, 1, 19), (19, 1, 18), (18, 1, 28), (28, 3,..."
497,498,"[(6, 1, 16), (16, 1, 26), (26, 3, 26), (26, 3,..."
498,499,"[(8, 1, 18), (18, 1, 28), (28, 3, 27), (27, 3,..."


In [6]:
world_trajGen.ShowTrajs_Learner()